In [1]:
import re
import json
import locale
import requests
import importlib
import pandas as pd
from mplogger import Logger
from datetime import datetime
from bs4 import BeautifulSoup
from collections import Counter
from html2tei import parse_date
from html2tei import parse_date
from warcio.archiveiterator import ArchiveIterator
from webarticlecurator import WarcCachingDownloader


import origo_specific

In [2]:


importlib.reload(origo_specific)
from origo_specific import get_meta_from_articles_spec as get_meta

## NOTES
- Ezzekkel a galéria cikkekkel mi lesz? https://www.origo.hu/auto/galeria/20220317-volkswagen-t7-multivan-etsi-teszt-2022-galeria.html
- https://www.origo.hu/sport/kozvetites/20211009-magyarorszag-albania-vbselejtezo-elo-kozvetites-puskas-arena-kepek-percrol-percre.html Formátum 3
- Miért rossz az olimpiás? https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html
- A közvetítésnél mi legyen az sch:name? 
- Archivum url-ek miért vannak benne az articles.warc-ban? https://www.origo.hu/hir-archivum/index.html -- BLACKLIST-elni kell őket
- Mi van a valasztas 2010-el?

## TODO
- Move keyword extraction from script tag outside and append extra keywords if found [DONE]

In [5]:
logger = Logger()

2022-07-26 11:24:26,780 INFO: Logging started


In [17]:
# Links:
kozvetites_link = 'https://www.origo.hu/sport/kozvetites/20211202-kezilabda-noi-vilagbajnoksag-magyarorszag-szlovakia-elo-kozvetites.html'
olimpia_link = 'https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html'

# format 1
auto_link = 'https://www.origo.hu/auto/20180523-maris-kitiltjak-a-regi-dizelautokat-egy-nemet-varosbol.html'
teches_format1 = 'https://www.origo.hu/techbazis/20080321-bittorrentes-rivalist-kap-az-apple-tv.html'
# format 2
ronaldo_link = 'https://www.origo.hu/sport/futball/20220315-ronaldo-ismet-bizonyithat-kedvenc-ellenfele-ellen-otthon-juthat-tovabb-a-bl-titkos-eselyese.html'

# format 3
gallery_link = 'https://www.origo.hu/sport/galeria/20220317-a-szexi-luisinha-barosa-oliveira-mclaren-lando-norris-kepek-fotok-galeria.html'


root_problem = 'https://www.origo.hu/auto/20150904-telekocsi-migrans-mav-vonat.html'
article_center = 'https://www.origo.hu/gazdasag/hirek/20091207-csokkenessel-nyitottak-a-new-yorki-tozsdek.html'
torlendo = 'https://www.origo.hu/gazdasag/hirek/20001011torlend1.html'
old_format = 'https://www.origo.hu/amerikai-elnokvalasztas-2008/index.html'

# ???
valasztas2010 = 'https://www.origo.hu/itthon/valasztas2010/20100210-ujabb-feltort-levelszekrenyek-utan-nyomoz-a-rendorseg.html'

palyabea = 'https://www.origo.hu/palyabea/post/20100909-elso-proba-a-nagykoncertre.html'
root_problem = 'https://www.origo.hu/itthon/onkormanyzati-valasztas-2010/hirek/20100930-polgar-bunpartolassal-vadolja-az-ugyeszseg-az-mszp-s-polgarmestert.html'
link_list = [kozvetites_link, olimpia_link, auto_link, teches_format1, ronaldo_link, gallery_link, root_problem, article_center, torlendo, old_format, valasztas2010, palyabea]

In [3]:
def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def extract_resp_record_data(resp):
    """Extract response date, WARC ID and raw HTML from a WARC response record"""
    warc_response_date = resp.rec_headers.get_header('WARC-Date')
    if '.' in warc_response_date:
        date_format = '%Y-%m-%dT%H:%M:%S.%fZ'
    else:
        date_format = '%Y-%m-%dT%H:%M:%SZ'
    warc_response_datetime = datetime.strptime(warc_response_date, date_format)
    warc_id = resp.rec_headers.get_header('WARC-Record-ID')
    raw_html = resp.content_stream().read().decode(resp.rec_headers.get_header('WARC-X-Detected-Encoding'))

    return warc_response_datetime, warc_id, raw_html

def from_request(link):
    txt = requests.get(link).text
    soup = BeautifulSoup(txt, 'lxml')
    return soup

def from_warc(warc, link):
    _, _, resp = warc.get_records(link)
    warc_response_datetime, warc_id, raw_html = extract_resp_record_data(resp)
    soup = BeautifulSoup(raw_html, 'html.parser')
    return soup


Keywords finding code

In [47]:
keywords_script = [a for a in from_request(ronaldo_link).find_all('script') if 'window.exclusionTags' in a.text][0].text
i1 = keywords_script.find('[')
i2 = keywords_script.find(']')
keywords_script[i1+1:i2].replace("'", '').split(',')

['labdarúgás',
 'sportműsor',
 'Manchester United FC',
 'Ajax',
 'Benfica',
 'Atlético Madrid',
 'Bajnokok Ligája']

ARCHIVE TESTING 

In [6]:
warc_filename = '../../warcs_dir/origo-articles_new9.warc.gz'
warc = WarcCachingDownloader(warc_filename, None, logger, just_cache=True, download_params={'stay_offline': True})

2022-07-26 11:24:31,073 INFO: Creating index for ../../warcs_dir/origo-articles_new9.warc.gz...


In [10]:
test_url='https://www.origo.hu/itthon/20100118-gaz-es-villanyorak-atallitasaval-gyanusitanak-negy-embert-heves-megyeben.html'

In [25]:
ARTICLE_ROOT_PARAMS_SPEC = [(('div',),{'class': 'col-xl-8'}),
                            (('div',), {'class': 'swiper-wrapper'}),
                            (('article',), {'id': 'article-center'}),
                            (('article',), {'id': 'article-text'}),
                            (('div',), {'id': 'cikk'}),
                            (('div',), {'class': 'article_text'}),
                            (('div',), {'id': 'article-text'}),
                            (('div',), {'id': {'kenyer-szov'}})
                           #(('div',), {'class': 'o-section-main'})
                           ]

Link testing

In [23]:

warc_filename = '../../warcs_dir/origo_root_problems.warc.gz'
w = WarcCachingDownloader(warc_filename, None, logger, just_cache=True, download_params={'stay_offline': True})

2022-04-12 23:58:33,041 INFO: Creating index for ../../warcs_dir/origo_root_problems.warc.gz...
2022-04-12 23:58:33,047 INFO: Index successfully created.


In [42]:
print(list(w.url_index)[3])
for f in ARTICLE_ROOT_PARAMS_SPEC:
    if from_warc(w, list(w.url_index)[0]).find(f[0], f[1]) is not None:
        print(f)

https://www.origo.hu/itthon/valasztas2010/kampanynaplo/20100129-kilep-az-mdfbol-a-mosonmagyarovari-polgarmester.html


In [36]:
from_warc(w, list(w.url_index)[1]).find_all('p')

[<p>2022.02.25. <span>péntek</span></p>]

In [43]:
with open('20100129-kilep-az-mdfbol-a-mosonmagyarovari-polgarmester.html', 'w') as htmlfile:
    htmlfile.write(str(from_warc(w, list(w.url_index)[3])))

In [10]:

warc_filename = '../../warcs_dir/origo-articles_new5.warc.gz'
w = WarcCachingDownloader(warc_filename, None, logger, just_cache=True, download_params={'stay_offline': True})

2022-04-12 12:52:20,989 INFO: Creating index for ../../warcs_dir/origo-articles_new5.warc.gz...
2022-04-12 12:58:17,016 INFO: Index successfully created.


In [6]:
# Unique link list
uniques = set([u.split('/')[3] for u in w.url_index])
unique_dict = {u:None for u in uniques}
for l in w.url_index:
    if unique_dict[l.split('/')[3]] == None:
        unique_dict[l.split('/')[3]] = l

In [17]:
del unique_dict['archivum']
unique_dict

{'uzletinegyed': 'https://www.origo.hu/uzletinegyed/hirdetes/kavosz/20091218-akik-sikert-kovacsoltak-a-szechenyi-kartyabol.html',
 'gazdasag': 'https://www.origo.hu/gazdasag/hirek/20090908-mariann-fischer-boel-eu-fobiztos-szamara-irt-levelet-a.html',
 'nagyvilag': 'https://www.origo.hu/nagyvilag/20190328-hatosagi-felugyelet-ala-helyezhettek-romaniaban-laura-corduta-kovesit.html',
 'techbazis': 'https://www.origo.hu/techbazis/20180828-harom-uj-iphone-erkezik.html',
 'jog': 'https://www.origo.hu/jog/20090324-mi-a-teendo-katyukar-eseten.html',
 'idojaras': 'https://www.origo.hu/idojaras/20130226-elaztatja-az-eso-a-nyugatdunantult-vas-zala-20-millimeter-eso.html',
 'teve': 'https://www.origo.hu/teve/19981211mk.html',
 'palyabea': 'https://www.origo.hu/palyabea/post/20110103-szabad-asszociacio-egy-keprol.html',
 'tudomany': 'https://www.origo.hu/tudomany/20171020-szaz-eve-kezdodott-a-caporettoi-attores.html',
 'filmklub': 'https://www.origo.hu/filmklub/20041217megszuletett.html',
 'itthon':

In [15]:
[u for u in w.url_index if 'bunpartolassal-vadolja' in u][0]

'https://www.origo.hu/itthon/onkormanyzati-valasztas-2010/hirek/20100930-polgar-bunpartolassal-vadolja-az-ugyeszseg-az-mszp-s-polgarmestert.html'

In [16]:
root_problem

'www.origo.hu/itthon/onkormanyzati-valasztas-2010/hirek/20100930-polgar-bunpartolassal-vadolja-az-ugyeszseg-az-mszp-s-polgarmestert.html'

In [18]:
get_meta(logger, root_problem, from_warc(w, root_problem))

defaultdict(<function html2tei.tei_utils.tei_defaultdict.<locals>.<lambda>()>,
            {'sch:url': 'https://www.origo.hu/itthon/onkormanyzati-valasztas-2010/hirek/20100930-polgar-bunpartolassal-vadolja-az-ugyeszseg-az-mszp-s-polgarmestert.html',
             'sch:name': 'Bűnpártolással vádol az ügyészség egy MSZP-s polgármestert',
             'sch:source': ['MTI'],
             'sch:datePublished': datetime.datetime(2010, 9, 30, 19, 24),
             'sch:dateModified': datetime.datetime(2010, 9, 30, 0, 0),
             'sch:keywords': ['hírek', 'önkormányzati választás 2010'],
             'sch:articleSection': 'Itthon'})

In [63]:
for v in unique_dict.values():
    print(get_meta(logger, v, from_warc(w, v)))

defaultdict(<function tei_defaultdict.<locals>.<lambda> at 0x7fc75a09d700>, {'sch:url': 'https://www.origo.hu/uzletinegyed/hirdetes/kavosz/20091218-akik-sikert-kovacsoltak-a-szechenyi-kartyabol.html', 'sch:name': 'Akik sikert kovácsoltak a Széchenyi kártyából', 'sch:authors': ['-'], 'sch:datePublished': datetime.datetime(2009, 12, 18, 8, 48), 'sch:dateModified': datetime.datetime(2009, 12, 18, 0, 0), 'sch:keywords': ['hirdetés'], 'sch:articleSection': 'Üzletinegyed'})
defaultdict(<function tei_defaultdict.<locals>.<lambda> at 0x7fc75a06ca60>, {'sch:url': 'https://www.origo.hu/gazdasag/hirek/20090908-mariann-fischer-boel-eu-fobiztos-szamara-irt-levelet-a.html', 'sch:name': 'Az EU-főbiztoshoz fordult a Magyar Agrárkamara', 'sch:source': ['MTI'], 'sch:datePublished': datetime.datetime(2009, 9, 8, 20, 10), 'sch:dateModified': datetime.datetime(2009, 9, 8, 0, 0), 'sch:keywords': ['agrárium', 'Magyar Agrárkamara', 'Mariann Fischer Boel', 'Európai Unió', 'hírek', 'agrárárak', 'EU', 'gabona'],

2022-04-12 10:58:52,690 WARNING: https://www.origo.hu/nepszavazas-2008/blog/20080304-nepszavazasi-ertesitoket-egetett-el-egy-postas.html ARTICLE NAME NOT FOUND!
2022-04-12 10:58:52,690 WARNING: https://www.origo.hu/nepszavazas-2008/blog/20080304-nepszavazasi-ertesitoket-egetett-el-egy-postas.html ARTICLE NAME NOT FOUND!
2022-04-12 10:58:52,692 WARNING: https://www.origo.hu/nepszavazas-2008/blog/20080304-nepszavazasi-ertesitoket-egetett-el-egy-postas.html FAILED TO FIND PUBLICATION DATE TAG!
2022-04-12 10:58:52,692 WARNING: https://www.origo.hu/nepszavazas-2008/blog/20080304-nepszavazasi-ertesitoket-egetett-el-egy-postas.html FAILED TO FIND PUBLICATION DATE TAG!


defaultdict(<function tei_defaultdict.<locals>.<lambda> at 0x7fc75ae328b0>, {'sch:url': 'https://www.origo.hu/tudomany/20171020-szaz-eve-kezdodott-a-caporettoi-attores.html', 'sch:name': 'Rengeteg magyar adta életét egy haszontalan győzelemért', 'sch:author': ['Origo'], 'sch:datePublished': datetime.datetime(2017, 10, 20, 17, 8), 'sch:dateModified': datetime.datetime(2017, 10, 20, 0, 0), 'sch:keywords': [' évforduló', ' áttörés', 'történelem', 'évforduló', 'I. világháború', 'Olaszország', ' I. világháború', ' Isonzo-folyó', 'áttörés', 'Osztrák-Magyar Monarchia', 'Isonzo-folyó', ' Osztrák-Magyar Monarchia', 'Tudomány', ' történelem', 'világháború', ' Olaszország'], 'sch:articleSection': 'Tudomány'})
defaultdict(<function tei_defaultdict.<locals>.<lambda> at 0x7fc75a1dc310>, {'sch:url': 'https://www.origo.hu/filmklub/20041217megszuletett.html', 'sch:name': 'Megszületett Gollam diagnózisa', 'sch:author': ['[origo]'], 'sch:datePublished': datetime.datetime(2004, 12, 17, 15, 51), 'sch:dateM

In [20]:
from_request('https://www.origo.hu/itthon/valasztas2010/felmeresek/20100210-nezopont-intezet-orbanhoz-vandorolnak-gyurcsany-szimpatizansai.html').find('div', {'id': 'cikk'})

<div id="cikk">
<!-- eTarget ContextAd Start -->
<script type="text/javascript">
     // <![CDATA[
       var sysuri = "/itthon/valasztas2010/felmeresek/20100210-nezopont-intezet-orbanhoz-vandorolnak-gyurcsany-szimpatizansai.html"
       var logoText = ""
       var logoLink = ""
       var logoImg = "none"
     // ]]>
     </script><script type="text/javascript">
		// <![CDATA[
			function SendToFriend() {
				var MyTemp;
				var MySysuri;
				
				if (location.href.indexOf('http://origo.hu/') > -1 || location.href.indexOf('http://www.origo.hu/') > -1) {
					MySysuri = sysuri;
					MyTemp = sysuri.substr(1, sysuri.indexOf('/', 1) - 1);
				} else {
					MySysuri = location.href;
					if (location.href.indexOf('http://www.klikkbank.hu/') > -1) {
					} else {
						if (MySysuri.indexOf('www.') > -1){
							MySysuri = MySysuri.replace(/www./g, '')
						}
					}
					MyTemp = MySysuri.substring(7, MySysuri.indexOf('.'));
					if (MySysuri.indexOf('klikkbank') > -1) {
						MyTemp = 'b

In [21]:
from_request('https://www.origo.hu/amerikai-elnokvalasztas-2008/blog/kampany/20081017-joe-a-vizvezetekszerelo-es-john-mccain-es-barack-obama-adopolitikaja.html').find('div', {'id': 'cikk'})

<div id="cikk" xmlns:msxsl="urn:schemas-microsoft-com:xslt" xmlns:user="http://mycompany.com/mynamespace">
<h1 class="cikk-cim">Befutott McCain vízvezetékszerelője</h1><span class="cikk-szerzo">Pethő András</span><span class="elvalaszt">|</span><span class="cikk-datum">2008. 10. 18., 12:46</span><span class="elvalaszt">|</span><div id="kommentszam"></div>
<div id="cimke-input"></div><script type="text/javascript">getCikkCimkek()</script><div class="archiv-cikk-box" id="archiv-cikk-box" style="display:none"></div><script type="text/javascript"> 
			var strModDate = '2008. 10. 18., 12:46';
			if (strModDate.substr(16,1) == ':') strModDate = strModDate.substr(0,15) + '0' + strModDate.substr(15,strModDate.length);
			var strNowDate = '2022. 04. 12., 13:09';
			
			var diffYears;
			//var diffMonths;
			var diffDays;
			
			function dateDiff() {
				date1 = new Date();
				date2 = new Date();
				diff  = new Date();
				
				date1temp = new Date(strModDate.substr(0,4),(strModDate.substr(6,2

KNOWN BAD and BLACKLISTED link testing

In [17]:
with open('origo_BLACKLIST.txt', 'r') as infile:
    blacklisted = infile.read().splitlines()

In [19]:
for b in w.url_index:
    if '' in b and b not in blacklisted:
        print(b)

In [ ]:
with open('known_bad_urls.txt', 'r') as infile:
    known_bad = infile.read().splitlines() 
not_bad = []
in_word = []
for link in known_bad:
    if 'archiv' in link:
        link = link.replace('/archiv/', '/teve/')

        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        if soup.find('div', {'class': 'nwm-404'}) is not None:
            print(link)
            not_bad.append(link)

        i1 = link.find('-')
        i2 = link[i1:].find('-')
        find_word = link[i1:i1+i2]
        if find_word not in soup.get_text():
            in_word.append(link)


In [57]:
not_tv = []
for link in blacklisted:
    link = link.replace('/archivum/', '/teve/')

    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('div', {'class': 'nwm-404'}) is not None:
        print(link)
        not_tv.append(link)


https://www.origo.hu/teve/20020812adallas.xml.html


In [59]:
blacklisted[0]

'https://www.origo.hu/archivum/20131022-kilencmilliot-vittek-el-gundel-takacs-musorabol.html'

In [60]:
not_tv_2 = []
for link in blacklisted:
    link = link.replace('/archivum/', '/teve/')

    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
known_bad_list


writing out dictionary python file

In [7]:
with open('origo_usable_urls.tsv', 'r') as tsvin, open('origo_usable_links_dict.txt', 'w') as txtout:
    link_dict = {c.split('\t')[1]:c.split('\t')[2] for c in tsvin.read().splitlines()}
    print('ORIGO_URL_CHANGES =\n', file=txtout)
    print(str(link_dict), file=txtout)





In [5]:
a = {'a': 'MISI', 
'b': 'FERI'}
print(str(a))

{'a': 'MISI', 'b': 'FERI'}


In [6]:
with open('dummy.txt', 'w') as dummy:
    dummy.write(str(a))

Check palyabea

In [25]:
from_request(palyabea).find('body', {'id':'!BODY_ID'}).find('div', {'id':'left'}).find_all('a', {'href':True, 'title':True, 'class':False})

[<a href="https://cimkezes.origo.hu/cimkek/lamm-david/index.html?tag=Lamm+D%E1vid" title="Lamm Dávid">Lamm Dávid</a>,
 <a href="https://cimkezes.origo.hu/cimkek/barcza-horvath-janos/index.html?tag=Barcza+Horv%E1th+J%E1nos" title="Barcza Horváth János">Barcza Horváth János</a>,
 <a href="https://cimkezes.origo.hu/cimkek/palya-bea/index.html?tag=Palya+Bea" title="Palya Bea">Palya Bea</a>,
 <a href="https://cimkezes.origo.hu/cimkek/des-andras/index.html?tag=D%E9s+Andr%E1s" title="Dés András">Dés András</a>,
 <a href="https://cimkezes.origo.hu/cimkek/budapest/index.html?tag=Budapest" title="Budapest">Budapest</a>,
 <a href="https://cimkezes.origo.hu/cimkek/koncert/index.html?tag=koncert" title="koncert">koncert</a>,
 <a href="https://cimkezes.origo.hu/cimkek/proba/index.html?tag=pr%F3ba" title="próba">próba</a>]

In [22]:
pubdate = from_request(palyabea).find('body', {'id':'!BODY_ID'}).find('div', {'class': 'date'})
parse_date(pubdate.get_text(strip=True), "%Y. %B. %e.")

In [15]:
# format 1 bs.find('header', class_='article-head')
# format 2 bs.find('header', {'id': 'article-head'})
# format 3 bs.find('body', {'class': 'gallery'})
# format 4 bs.find('div', class_='sportonline_header')
# format 5 bs.find('div', {'id': 'cikk'})
for c, l in enumerate(link_list):
    bs = from_request(l)
    if bs.find('header', class_='article-head') is not None: print(1, l)
    if bs.find('header', {'id': 'article-head'}) is not None: print(2, l)
    if bs.find('body', {'class': 'gallery'}) is not None: print(3, l)
    if bs.find('div', class_='sportonline_header') is not None: print(4, l)
    if bs.find('div', {'id': 'cikk'}) is not None: print(5, l)
    if bs.find('body', {'id':'!BODY_ID'}) is not None: print(6, l)

4 https://www.origo.hu/sport/kozvetites/20211202-kezilabda-noi-vilagbajnoksag-magyarorszag-szlovakia-elo-kozvetites.html
3 https://www.origo.hu/sport/olimpia/galeria/20210726-gordeszka-latvanyos-kepek.html
1 https://www.origo.hu/auto/20180523-maris-kitiltjak-a-regi-dizelautokat-egy-nemet-varosbol.html
1 https://www.origo.hu/techbazis/20080321-bittorrentes-rivalist-kap-az-apple-tv.html
2 https://www.origo.hu/sport/futball/20220315-ronaldo-ismet-bizonyithat-kedvenc-ellenfele-ellen-otthon-juthat-tovabb-a-bl-titkos-eselyese.html
3 https://www.origo.hu/sport/galeria/20220317-a-szexi-luisinha-barosa-oliveira-mclaren-lando-norris-kepek-fotok-galeria.html
1 https://www.origo.hu/auto/20150904-telekocsi-migrans-mav-vonat.html
2 https://www.origo.hu/gazdasag/hirek/20091207-csokkenessel-nyitottak-a-new-yorki-tozsdek.html
2 https://www.origo.hu/gazdasag/hirek/20001011torlend1.html
5 https://www.origo.hu/itthon/valasztas2010/20100210-ujabb-feltort-levelszekrenyek-utan-nyomoz-a-rendorseg.html
6 https

Meta json

In [26]:
a = [1,2,3,4,5]

In [30]:
if len(a) >= 5+1 and a[5] > 2:
    print('he')